# Project 4: Lid Driven Cavity


- Resource: https://github.com/davidlin001/flow_sims/tree/master/SIMPLE


`Libraries`


In [4]:
import numpy as np
import matplotlib.pyplot as plt
from solvers import *

`Initial Variables`


In [ ]:
Nx = Ny = 100
l = 1.0
nu = 0.01
velocity = 1.0
Re = 1.0

dx = l / Nx
dy = l / Ny

x = np.linspace(0.0, l, Nx)
y = np.linspace(0.0, l, Ny)
X,Y = np.meshgrid(x,y)

u = u_star = np.zeros((Nx+2, Ny+2))
v = v_star =  np.zeros((Nx+2,Ny + 2))
p = p_star = p_prime = np.zeros((Nx+2, Ny+2))

`Coefficients`


In [ ]:
A_p = A_w = A_s = A_e = A_n = np.zeros((Nx+2, Ny+2))
Ap_p = Ap_w = Ap_s = Ap_e = Ap_n = np.zeros((Nx+2, Ny+2))

source_x = np.zeros(Nx*Ny) 
source_y = np.zeros(Nx*Ny) 
source_p = np.zeros(Nx*Ny) 
u_face = np.zeros((Ny+2,Nx+1))
v_face = np.zeros((Ny+1,Nx+2))

A_momentum = np.zeros((Nx*Ny, Nx*Ny))
A_pressure = np.zeros((Nx*Ny, Nx*Ny))

`Boundry Conditions`


In [ ]:
u[0, 1:-1] = 1

In [ ]:
while True:
    u_star, du = x_momentum_solver(Nx,Ny, dx, dy, u, v, velocity,p)
    v_star, dv= y_momentum_solver(Nx, Ny, dx, dy, u, v, velocity,p)
    p,p_prime = pressure_correction(Nx,Ny,dx,dy,u_star,v_star,p)
    u,v = update_velocity(Nx, Ny, u_star, v_star, p_prime, du, dv, velocity)

f:\M.Tech\SEM 1\Computational Fluid Dynamics\Projects\Project4\solvers.py:15: RuntimeWarning: invalid value encountered in scalar divide
  factor = l[i-1] / d[i-1]


[[ 0. nan nan ... nan nan  1.]
 [ 0. nan nan ... nan nan  1.]
 [ 0. nan nan ... nan nan  1.]
 ...
 [ 0. nan nan ... nan nan  1.]
 [ 0. nan nan ... nan nan  1.]
 [ 0. nan nan ... nan nan  1.]]
[[ 0. nan nan ... nan nan  1.]
 [ 0. nan nan ... nan nan  1.]
 [ 0. nan nan ... nan nan  1.]
 ...
 [ 0. nan nan ... nan nan  1.]
 [ 0. nan nan ... nan nan  1.]
 [ 0. nan nan ... nan nan  1.]]


KeyboardInterrupt: 